In [1]:
%matplotlib widget

from collections import deque
from copy import deepcopy
from pathlib import Path
from random import shuffle

import flammkuchen as fl
import numpy as np
import pandas as pd
from bg_atlasapi import BrainGlobeAtlas
from bg_atlasapi.core import Atlas
from matplotlib import pyplot as plt
from motions.em import NeuroSkeleton, load_skeletons_from_zip
from motions.em.skeleton_mesh import make_full_neuron
from scipy.ndimage import gaussian_filter
from tifffile import imread
from tqdm import tqdm

atlas = Atlas("/Users/luigipetrucco/.brainglobe/ipn_zfish_0.5um_v1.4")

from numba import jit, prange


@jit(nopython=True, parallel=True)
def _set_voxels(stack, idxs):
    for i in prange(idxs.shape[0]):
        stack[idxs[i, 0], idxs[i, 1], idxs[i, 2]] += 1


import lotr.plotting as pltltr

COLS = pltltr.COLS

# Neuron plotter
from motions.em.plotting import AtlasPlotter

plotter = AtlasPlotter(
    structures=["lat_l_col", "ipn", "lat_r_col", "med_l_col", "med_r_col"]  #  ipn_somas
)
# plotter = AtlasPlotter(structures=["ipn"])
plotter.shape_color = None
plotter.shape_fill = False
plotter.shape_linewidth = 0.2
plotter.shape_linecolor = (0.1,) * 3
plotter.shape_alpha = 1
plotter.font_size = 10
plotter.custom_switch = dict(sagittal=0, frontal=1, horizontal=0)

ModuleNotFoundError: No module named 'motions'

In [ ]:
master_folder = Path(r"J:\_Shared\experiments\E0060_ipn_em_tracing")
# master_folder =  Path(r"C:\Users\lpetrucco\Desktop")
master_folder = Path("/Users/luigipetrucco/Google Drive/data/ipn_tracing/")
data_folder = master_folder / "main_data_dir"
neurons_dict = fl.load(data_folder / "all_skeletons.h5")

working_folder = master_folder / "rosette"
rosette_volume = fl.load(working_folder / "volume.h5")
with open(working_folder / "rosette_ids.txt", "r") as file:
    rosette_ids = file.read().splitlines()

In [ ]:
stack = imread(
    r"/Volumes/Shared/confocal/gad1b_gal4_6s_huc_h2b_mcherry/201002_f0_gad1b-gcamo6s_h2b-mcherry_2020_10_02__12_21_33.lsm"
)
# stack = imread(r"/Volumes/Shared/confocal/gad1b_gal4_6s_huc_h2b_mcherry/201002_f3_gad1b-gcamo6s_h2b-mcherry_2020_10_02__16_21_08.lsm")

In [ ]:
stack_mn = stack[:, 1, 180:320, :].mean(1)

for i in range(stack_mn.shape[0]):
    stack_mn[i, :] = stack_mn[i, :]
    stack_mn[i, :] = stack_mn[i, :] - np.mean(
        stack_mn[i, stack_mn[i, :] < np.percentile(stack_mn[i, :], 80)]
    )

stack_mn = stack_mn - np.min(stack_mn)
anatomy = stack_mn

In [ ]:
plt.figure()
plt.imshow(stack_mn ** 0.9)

In [ ]:
r_hab_ids = [k for k, s in neurons_dict.items() if "rhab" in s.cell_comments]
l_hab_ids = [k for k, s in neurons_dict.items() if "lhab" in s.cell_comments]

hab_ids = set(r_hab_ids + l_hab_ids)
ahb_ids = set([k for k, s in neurons_dict.items() if "rf" in s.cell_comments])
ipn_ids = set(neurons_dict.keys()) - hab_ids - ahb_ids

k_dict = dict()
for k, s in neurons_dict.items():
    if k in hab_ids:
        k_dict[k] = "hab"
    elif k in ahb_ids:
        k_dict[k] = "ahb"
    elif k in ipn_ids:
        k_dict[k] = "ipn"

# Plot all aHB neurons

In [ ]:
plotter = AtlasPlotter(
    structures=["lat_l_col", "ipn", "lat_r_col", "med_l_col", "med_r_col"]  #  ipn_somas
)
plotter = AtlasPlotter(structures=["ipn"])  #  ipn_somas
plotter.shape_color = None
plotter.shape_fill = False
plotter.shape_linewidth = 0.2
plotter.shape_linecolor = (0.1,) * 3
plotter.shape_alpha = 1
plotter.font_size = 10
plotter.custom_switch = dict(sagittal=0, frontal=1, horizontal=0)

In [ ]:
plotter.custom_switch = None

f, axs_all = plt.subplots(2, 3, figsize=(7, 4))
# f = plt.figure(figsize=(10, 6))
# ax0 = f.add_axes((0.3, 0., 0.7, 0.46))
# ax1 = f.add_axes((0.3, 0.5, 0.7, 0.46))
# ax2 = f.add_axes((0.0, 0.5, 0.0001, 0.0001))
# axs = np.array([ax2, ax0, ax1])

plotter.generate_projection_plots(axs_all[0], labels=False, title=True, edge=250)
plotter.generate_projection_plots(axs_all[1], labels=True, title=False, edge=250)

# for cid in all_ks:
#    n_dict = neurons_dict[cid]

#    plotter.plot_neurons(axs, n_dict, color=(0.7,)*3, linewidth=0.3)
cols_dict = dict(hab=[0.8, 0.24, 0.35], ahb=[0.3, 0.49, 0.85], ipn=[0.49, 0.8, 0.3])

# for i, (cids, col) in enumerate(zip([hab_ids, ahb_ids, ipn_ids],
#                                    )):
rasterized = True
cols = sns.color_palette()
ids = list(k_dict.keys())
shuffle(ids)
k = 0
for cid in ids:
    n_dict = neurons_dict[cid]
    if (
        k_dict[cid] == "ahb"
        and not "??" in n_dict.cell_comments
        and not "unknown" in n_dict.cell_comments
    ):

        print(n_dict.cell_comments)
        if "dipn" in n_dict.cell_comments:
            axs = axs_all[0]
        else:
            axs = axs_all[1]

        plotter.plot_neurons(
            axs,
            n_dict,
            color=cols[k],
            select="dendrites",
            linewidth=0.5,
            rasterized=rasterized,
        )
        try:
            plotter.plot_neurons(
                axs,
                n_dict,
                color=cols[k],
                select="axon",
                linewidth=0.5,
                rasterized=rasterized,
            )
        except ValueError:
            pass
        k += 1
        if k == len(cols):
            k = 0
axs_all[0][0].set_ylabel("dIPN")
axs_all[1][0].set_ylabel("vIPN")

# f.savefig(
#    "/Users/luigipetrucco/Desktop/friday_figures/ahb_neurons.pdf",
#    format="pdf",
#    transparent=True,
#    pad_inches=0.5,
#    dpi=300,
# )

In [ ]:
[int((1 - c) * 255) for c in cols[3]]

# Plot all aHB neurons with confocal

In [ ]:
plotter = AtlasPlotter(
    structures=["lat_l_col", "ipn", "lat_r_col", "med_l_col", "med_r_col"]  #  ipn_somas
)
plotter = AtlasPlotter(structures=["ipn"])  #  ipn_somas
plotter.shape_color = None
plotter.shape_fill = False
plotter.shape_linewidth = 1
plotter.shape_linecolor = (0.1,) * 3
plotter.shape_alpha = 1
plotter.font_size = 6
plotter.custom_switch = dict(sagittal=0, frontal=1, horizontal=0)

In [ ]:
f2, axs_dummy = plt.subplots(1, 2, figsize=(0.5, 0.4))

In [ ]:
for i, cid in enumerate(ids):
    n_dict = neurons_dict[cid]
    if (
        k_dict[cid] == "ahb"
        and not "??" in n_dict.cell_comments
        and not "unknown" in n_dict.cell_comments
    ):
        print(neurons_dict[cid].cell_comments)

In [ ]:
sel = [
    "p048",
    "p085",
    "p041",
    "p077",
    "p075",
    "p043",
    "p089",
    "p084",
    "p096",
    "p088",
    "p070",
    "p056",
]

In [ ]:
import seaborn as sns

sns.set(palette="colorblind")
cols = sns.color_palette()[:-1]

In [ ]:
len(cols)

In [ ]:
plotter.custom_switch = None

f, axs_true = plt.subplots(1, 1, figsize=(5, 4))
axs = np.array([axs_true,] + list(axs_dummy))
# f, axs = plt.subplots(1, 3, figsize=(5, 4))


plotter.shape_linewidth = 1
plotter.generate_projection_plots(axs, labels=True, title=True, edge=280)
rasterized = True
ids = list(k_dict.keys())
shuffle(ids)
k = 0
for i, cid in enumerate(ids):
    n_dict = neurons_dict[cid]
    if (
        k_dict[cid] == "ahb"
        and not "??" in n_dict.cell_comments
        and not "unknown" in n_dict.cell_comments
    ):
        plotter.plot_neurons(
            axs,
            n_dict,
            color=cols[
                i % 9
            ],  # COLS["fish_cols"][i % 10],  # pltltr.shift_lum(COLS["ring"], -0.25),
            select="dendrites",
            soma_s=30,
            linewidth=0.8,
            rasterized=rasterized,
            alpha=1,
        )
        try:
            plotter.plot_neurons(
                axs,
                n_dict,
                color=cols[i % 9],
                # pltltr.shift_lum(COLS["ring"], -0.25),
                select="axon",
                soma_s=30,
                linewidth=0.6,
                rasterized=rasterized,
                alpha=1,
            )
        except:
            pass

        k += 1

axs[0].imshow(
    stack_mn[68:, :] ** 0.8,
    extent=[150, -30, -140, 150],
    zorder=-100,
    cmap="gray_r",
    vmin=4,
    origin="upper",
)

pltltr.savefig("em_all_and_gad1b_stack_planes")

In [ ]:
78 * 0.5 * (150 / 140 - 1) / 3

In [ ]:
anatomy = stack[:, 1, :, :].copy()
# for i in range(anatomy.shape[0]):
# sel = anatomy[i, :, :]
# offset = np.mean(sel[sel < np.percentile(sel, 80)])
# anatomy[i, :, :] = anatomy[i, :, :] - np.mean(anatomy[i, :, :])

In [ ]:
f, axs = plt.subplots(4, 1, figsize=(2, 7))

for i, (plane, vlims) in enumerate(
    zip([290, 220, 150, 110], [None, None, (3, 50), (3, 60)])
):
    if vlims is None:
        axs[i].imshow(anatomy[plane, :, :], origin="upper", cmap="gray_r")
    else:
        axs[i].imshow(
            anatomy[plane, :, :],
            origin="upper",
            cmap="gray_r",
            vmin=vlims[0],
            vmax=vlims[1],
        )

    [axs[i].spines[k].set_visible(False) for k in ["left", "right", "top", "bottom"]]
    axs[i].set_xticks([])
    axs[i].set_yticks([])

    # axs.set_ylabel("Post - Ant")
    # axs.set_xlabel("Ventral - Dorsal")
    axs[i].axis("equal")
# plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.05, top=1, bottom=0, left=0, right=1)

# f.savefig(
#    "/Users/luigipetrucco/Desktop/friday_figures/ahb_neurons_planes.pdf",
#    format="pdf",
#    transparent=True,
#    pad_inches=0.5,
#    dpi=300,
# )

In [ ]:
plt.figure()
plt.imshow(anatomy.mean(1))

In [ ]:
plotter.custom_switch = None
edge = 140
f, axs = plt.subplots(1, 3, figsize=(7, 3))
# fdummy = plt.figure(figsize=(0.1, 0.1))
# ax_dummy = fdummy.add_axes((0, 0, 1, 1))
# axs = np.concatenate([axs, [ax_dummy]])

plotter.generate_projection_plots(axs, labels=True, title=True, edge=edge)

rasterized = True
cols = sns.color_palette()
ids = list(k_dict.keys())
shuffle(ids)
k = 0
for cid in ids:
    n_dict = neurons_dict[cid]
    if "p077" in n_dict.cell_comments or "p089" in n_dict.cell_comments:
        plotter.plot_neurons(
            axs,
            n_dict,
            color=pltltr.shift_lum(cols[k], 0.15),
            select="axon",
            linewidth=0.8,
            rasterized=rasterized,
        )
        plotter.plot_neurons(
            axs,
            n_dict,
            color=pltltr.shift_lum(cols[k], -0.15),
            select="dendrites",
            linewidth=0.8,
            rasterized=rasterized,
        )
        k += 1
        if k == len(cols):
            k = 0

axs_all[0][0].set_ylabel("dIPN")
axs_all[1][0].set_ylabel("vIPN")
pltltr.savefig(f"single_planar_neuron_e{edge}")

In [ ]:
plotter.custom_switch = None
edge = 140
f, axs = plt.subplots(1, 3, figsize=(7, 3))
# fdummy = plt.figure(figsize=(0.1, 0.1))
# ax_dummy = fdummy.add_axes((0, 0, 1, 1))
# axs = np.concatenate([axs, [ax_dummy]])

plotter.generate_projection_plots(axs, labels=True, title=True, edge=edge)

rasterized = True
cols = sns.color_palette()
ids = list(k_dict.keys())
shuffle(ids)
k = 0
for cid in ids:
    n_dict = neurons_dict[cid]
    if (
        k_dict[cid] == "ahb"
        and not "??" in n_dict.cell_comments
        and not "unknown" in n_dict.cell_comments
        and not "proj" in n_dict.cell_comments
        and "plan" in n_dict.cell_comments
    ):
        if "dipn" in n_dict.cell_comments:
            try:
                plotter.plot_neurons(
                    axs,
                    n_dict,
                    color=pltltr.shift_lum(cols[k], 0.15),
                    select="axon",
                    linewidth=0.8,
                    rasterized=rasterized,
                )
                plotter.plot_neurons(
                    axs,
                    n_dict,
                    color=pltltr.shift_lum(cols[k], -0.15),
                    select="dendrites",
                    linewidth=0.8,
                    rasterized=rasterized,
                )
                print(n_dict.cell_comments)
            except ValueError:
                pass
            k += 1
            if k == len(cols):
                k = 0
axs_all[0][0].set_ylabel("dIPN")
axs_all[1][0].set_ylabel("vIPN")
pltltr.savefig(f"dorsal_ipn_planar_dendrites_{edge}")